# RAG(Retrieval-Augmented Generation) based QA Chat bot

## 0. 미션
참조
- 정보: https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma?hl=ko
- 2b: https://huggingface.co/google/gemma-2b
- 2b instruction tuning: https://huggingface.co/google/gemma-1.1-2b-it
- 7b: https://huggingface.co/google/gemma-7b
- 7b instruction tuning: https://huggingface.co/google/gemma-1.1-7b-it
- BM25: https://github.com/dorianbrown/rank_bm25
- SentenceTransformers: https://www.sbert.net/

미션
- 질문에 대해서 적절한 문서를 검색하고, 검색된 문서에 근거해서 답변하는 RAG 챕봇을 만들어봅니다.
- 문서 검색은 2일차 실습 결과를 사용합니다.
- 검색된 문서에 근거해서 답변하는 기능은 구글의 gemma-2b-it SLLM을 사용합니다.
- 필요에 따라서 gemma-2b-it를 fine-tuning 합니다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [ ]:
!pip install -q -U transformers==4.38.2
!pip install -q -U datasets==2.18.0
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.9.0
!pip install -q -U trl==0.7.11
!pip install -q -U accelerate==0.27.2
!pip install -q -U rank_bm25==0.2.2
!pip install -q -U sentence-transformers==2.7.0
!pip install -q -U wikiextractor==3.0.6
!pip install -q -U konlpy==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [ ]:
import os
import glob
import json

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import konlpy
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

### 3.2. HuggingFace login
- 이번 프로젝트는 HuggingFace 로그인 해야만 진행이 가능합니다.
- HuggingFace 계정이 없다면 아래 URL에 접속해서 가입하시기 바랍니다.
  - https://huggingface.co/
- HuggingFace 로그인을 위해서 아래 URL에 접속해서 'User Access Token'을 생성하고 복사해서 Token에 입력하세요.
  - https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# access token을 복사하세요.
HF_TOKEN = "ACESS_TOKEN"

### 3.3. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- SEARCH_MODEL_ID
  - 검색을 위한 SentenceTransformer 입니다.
  - 서울대학교 컴퓨터언어학_자연어처리 연구실에서 공개한 모델입니다.
  - https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
- SLLM_MODEL_ID
  - 문서에 근거해서 답변 기능을 위한 SLLM 입니다.
  - 구글에서 공개한 gemma-2b를 Instruction tunned한 버전입니다.
  - https://huggingface.co/google/gemma-2b-it
- CHUNK_FN
  - 문서를 일정한 단위로 분할해서 저장할 파일 이름
  - 데이터베이스를 대신하는 역할
  - 실제 기능을 만들 때는 DB를 사용해야 합니다.


In [ ]:
WORKSPACE = '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week10/Mini_project6/LLM/nlp-project'
SEARCH_MODEL_ID = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
SLLM_MODEL_ID = 'google/gemma-1.1-2b-it'
SLLM_MODEL_ID2 = 'google/gemma-1.1-7b-it'
CHUNK_FN = os.path.join(WORKSPACE, "data", "chunk_db.json")

## 4. SLLM RAG tutorial (재시작 필요)
- SLLM에 질문과 근거 문서를 함께 입력하고 질문에 맞는 답변을 근거 문서로 부터 하도록 하는 과정을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 4.1. model load with 4 bits
- 2B token을 가진 gemma를 그냥 로딩할 경우는 약 9G의 GPU vRAM이 필요합니다.
- 4bit 양자화를 할 경우 2.2G의 GPU vRAM 필요.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

### 4.2. pipeline
- https://huggingface.co/docs/transformers/main_classes/pipelines
- huggingface에서 inference를 쉽게 하기 위해 정의한 라이브러리.

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

### 4.3. sllm prompt
- https://huggingface.co/google/gemma-1.1-2b-it
- 아래와 같은 형식이 gemma의 promt 형식 입니다.
```
<bos><start_of_turn>user
{content}<end_of_turn>
<start_of_turn>model
```
- NSMC 추론을 위한 프롬프트를 생성하는 과정입니다.

In [ ]:
query = "지미 카터 대통령이 졸업한 대학교는?"
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {chunk_list[4]}

문서4: {chunk_list[3]}

문서3: {chunk_list[2]}

문서2: {chunk_list[1]}

문서1: {chunk_list[0]}

질문: {query}"""
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
print(prompt)

<bos><start_of_turn>user
당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: 화학
화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.
어원.
화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.
‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.
역사.
고대 화학(古代化學)

문서4: 문학
문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.
일반적으로 문학

### 4.4. sllm inference
- 이전 단계에서 생성한 prompt를 이용해 추론하고 결과를 확인하는 과장입니다.

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
outputs

[{'generated_text': '<bos><start_of_turn>user\n당신이 가진 지식을 의존하지 말고 \'문서1\'부터 \'문서5\'를 참고해서 \'질문\'에 대해서 답변해 주세요.:\n\n문서5: 화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)\n\n문서4: 문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함

In [ ]:
print(outputs[0]["generated_text"])

<bos><start_of_turn>user
당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: 화학
화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.
어원.
화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.
‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.
역사.
고대 화학(古代化學)

문서4: 문학
문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.
일반적으로 문학

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

문서에서 지미 카터 대통령은 조지아 공과대학교를 졸업했습니다.


### 4.5. sllm chatbot
- chatbot 형식의 QA 예 입니다.

In [ ]:
# 프롬프트 생성 함수
def gen_prompt(pipe, chunk_list, query):
    messages = [
        {
            "role": "user",
            "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {chunk_list[4]}

문서4: {chunk_list[3]}

문서3: {chunk_list[2]}

문서2: {chunk_list[1]}

문서1: {chunk_list[0]}

질문: {query}"""
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = gen_response(pipe, chunk_list, query)
    print(f'답변 > {result}\n\n')

질문 > 수학 상수가 뭐야?
답변 > 수학 상수는 물리적 측정과 상관없이 정의된 불변량으로, 변수의 반대말이다.


질문 > 지미카터의 직책이 뭐야?
답변 > 질문에 대한 답변은 지미 카터의 직책이 도덕주의 정책을 내세는 것으로 알려져 있습니다.


질문 > 


## 5. SLLM no RAG 실습 (재시작 필요)
- RAG를 사용하지 않고 입력한 질문만을 이용해서 SLLM의 답변 능력을 확인하는 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 5.1. SLLM no RAG
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문을 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
# 프롬프트 생성 함수
def gen_prompt(pipe, query):
    messages = [
        {
            "role": "user",
            "content": f"""당신이 가진 지식을 참고해서 '질문'에 대해서 답변해 주세요.:

질문: {query}"""
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, query):
    prompt = gen_prompt(pipe, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = gen_response(pipe, query)
    print(f'답변 > {result}\n\n')

질문 > 수학이 뭐야?
답변 > **수학은 다음과 같은 개념과 원리를 포함하는 학문입니다.**

* **숫자와 관계:** 수학은 수와 관계를 이해하고 계산하는 것을 목표로 합니다.
* **추리:** 수학은 문제 해결과 새로운 것을 창작하는 것을 목표로 합니다.
* **통계:** 데이터를 분석하고 추론하는 것을 목표로 합니다.
* **미적분:** 변화율과 미래를 예측하는 것을 목표로 합니다.
* **통계:** 데이터를 분석하고 추론하는 것을 목표로 합니다.


질문 > 


## 6. SLLM RAG with BM25 실습 (재시작 필요)
- BM25의 검색 기능과 SLLM의 추론 기능을 합쳐 RAG를 구성하는 실습입니다.
- BM25에서 tokenizer를 사용합니다.
- SLLM에서도 tokenizer를 사용합니다.
- 두 tokeinzer에 각각 다른 구분할 수 있는 이름을 변수 명으로 선언해야 합니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 6.1. bm25를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.
- BM25를 위해서 tokenizer를 정의하고 전처리를 실행합니다.

In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [ ]:
# 빈 리스트 생성
data_list = []

# JSON 파일을 줄 단위로 읽어오기
with open(CHUNK_FN, 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 JSON 객체로 변환하여 리스트에 추가
        data = json.loads(line)
        data_list.append(data)

# 데이터 프레임으로 변환
df = pd.DataFrame(data_list)

chunk_list = df['chunk'].to_list()

# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in chunk_list]
print(tokenized_chunks[0])

['지미 카터', '제임스', '얼', '“', '지미', '”', '카터', '주니어', '(', ',', '1924', '년', '10월 1일', '~', ')', '민주당', '출신', '미국', '제', '39', '대', '대통령', '(', '1977', '-', '81', ')', '이', '.', '생애', '.', '어리', '시절', '.', '지미 카터', '조지', '아주', '섬터', '카운티', '플레인스', '마을', '태어나', '.', '조지아 공과대학교', '졸업', '하', '.', '그 후', '해군', '들어가', '전함', '·', '원자력', '·', '잠수함', '승무원', '일', '하', '.', '1953', '년', '미국 해군', '대위', '예편', '하', '이후', '땅콩', '·', '면', '화', '등', '가꾸', '많', '돈', '벌', '.', '그', '별명', '"', '땅콩', '농부', '"', '(', 'Peanut', 'Farmer', ')', '알리', '지', '.', '정계', '입문', '.', '1962', '년', '조지', '아주', '상원', '의원', '선거', '낙선', '하', ',', '그', '선거', '부정', '선거', '이', '입증', '하', '되', '당선', '되', ',', '1966', '년', '조지아 주', '지사', '선거', '낙선', '하', ',', '1970', '년', '조지아 주', '지사', '선거', '당선', '되', '.', '대통령', '되', '전', '조지', '아주', '상원', '의원', '두', '번', '연임', '하', ',', '1971', '년', '1975', '년', '조지아', '지사', '근무', '하', '.', '조지아 주', '지사', '지내', ',', '미국', '살', '흑인', '등용', '법', '내세우', '.', '대통령', '재임', '.', '1976', '년', '미', 

### 6.2. BM25 검색 함수
- bm25를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
bm25 = BM25Okapi(tokenized_chunks)

In [ ]:
def query_bm25(bm25, query, tokenizer, top_n=10):
    tokenized_query = tokenizer(query)
    # score 계산
    doc_scores = bm25.get_scores(tokenized_query)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        if doc_scores[i] > 0:
            result.append((i, doc_scores[i]))
    return result

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_bm25(bm25, query, tokenizer)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학
---- score: 5.530034825502101 ----
수학
역사.
역사적으로 고대부터 현대에 이르기까지 문명에 필수적인 건축, 천문학, 정치, 상업 등에 수학적 개념들이 응용되어 왔다. 교역·분배·과세 등 인류의 사회 생활에 필요한 모든 계산에 수학이 관여해 왔고, 농경 생활에 필수적인 천문 관측과 달력의 제정, 토지의 측량 또한 수학이 직접적으로 사용된 분야이다. 고대 수학을 크게 발전시킨 문명으로는 메소포타미아, 이집트, 인도, 중국, 그리스 등이 있다.
특히 고대 그리스 문명에서는 처음으로 방정식에서 변수를 문자로 쓰는 등 추상화가 발전하였고 유클리드의 원론에서는 최초로 엄밀한 논증에 근거한 수학이 나타난다. 수학의 발전은 이후로도 계속되어 16세기의 르네상스에 이르러서는 과학적 방법과의 상호 작용을 통해 수학과 자연과학에 있어서 혁명적인 연구들이 진척되었고, 이는 인류 문명 발달에 큰 영향을 미치게 되었다.
세부 분야.
수학의 각 분야들은 상업에 필요한 계산을 하기 위해, 숫자들의 관계를 이해하기 위해, 토지를 측량하기 위해, 그리고 천문학적 사건들을 예견하기 위해 발전되어왔다. 이 네 가지 목적은 대략적으로 수학이 다루는 대상인 양, 구조, 공간 및 변화에 대응되며, 이들을 다루는 수학의 분야를 각각 산술, 대수학, 기하학, 해석학이라 한다. 또한 이 밖에도 근대 이후에 나타난 수학기초론과 이산수학 및 응용수학 등이 있다.

---- score: 5.4672796502298695 ----
수학
수학기초론 관련 분야.
수학의 기초를 확실히 세우기 위해, 수리논리학과 집합론이 발전하였고, 이와 더불어 범주론이 최근에도 발전되고 있다. “근본 위기”라는 말은 대략 1900년에서 1930년 사이에 일어난, 수학의 엄밀한 기초에 대한 탐구를 상징적으로 보여주는 말이다. 수학의 엄밀한 기초에 대한 몇 가지 의견 불일치는 오늘날에도 계속되고 있다. 수학의 기초에 대한 위기는 그 당시 수많은 논쟁에 의해 촉발되었으며, 그 논쟁에는 칸토어의 집

### 6.3. SLLM RAG with BM25 구현
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문에 대해서 관련된 문서를 bm25를 이용해서 5개 구합니다.
  - bm25의 경우 응답이 5개 이하일 수 있습니다. 이 부분을 고려하세요.
  - 질문과 함께 문서 5개를 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
def gen_prompt(pipe, chunk_list, query):
    # 문서 정보 문자열 생성
    doc_info = "\n\n".join([f"문서{i + 1}: {doc}" for i, doc in enumerate(chunk_list)])

    # 질문 문자열 생성
    question_info = f"질문: {query}"

    # 사용자 메시지 생성
    user_message = {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 문서들을 참고해서 질문에 대해 답변해 주세요:\n\n{doc_info}\n\n{question_info}"""

    }

    # 프롬프트 생성
    prompt = pipe.tokenizer.apply_chat_template([user_message],
                                                tokenize=False,
                                                add_generation_prompt=True)

    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_bm25(bm25, query, tokenizer, top_n=5)
    chunk_list_result = [chunk_list[i] for i, score in result]
    result = gen_response(pipe, chunk_list_result, query)
    print(f'답변 > {result}\n\n')

검색 > 수학
답변 > **수학**은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문입니다. 수학은 현대 수학에서 중요한 역할을 하며, 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서 핵심적인 역할을 합니다.

**수학의 개념**

- 기원전 600년 경에 활동하며 최초의 수학자로 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있습니다.
- 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어집니다.
- 수학은 다른 학문들과 구별되는 특징을 가지고 있습니다.

**수학의 역사**

- 고대 수학은 메소포타미아, 이집트, 인도, 중국, 그리스 등의 문명에서 발전했습니다.
- 수학의 기초를 확실히 세우기 위해 수리논리학과 집합론이 발전했습니다.
- 16세기에 갈릴레오 갈릴레이는 "자연이라는 책은 수학이라는 언어로 기록되어 있다."는 주장과 함께 물리학에 수학적 방법을 도입했습니다.

**수학의 영향**

- 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서 중요한 도구로 사용됩니다.
- 수학은 미적분학과 선형대수학, 확률론과 같은 다른 수학 분야에도 큰 영향을 미했습니다.
- 수학은 또한 음악이나 미술 등 예술과도 관련이 있습니다.


검색 > 


## 7. SLLM RAG with SentenceTransformers 실습 (재시작 필요)
- SentenceTransformers의 검색 기능과 SLLM의 추론 기능을 합쳐 RAG를 구성하는 실습입니다.
- SentenceTransformers도 model을 사용합니다.
- SLLM도 model을 사용합니다.
- 두 model에 각각 다른 구분할 수 있는 이름을 변수 명으로 선언해야 합니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 7.1. SentenceTransformers를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.

In [ ]:
# SentenceTransformer 모델 생성
sentence_model = SentenceTransformer(SEARCH_MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# chunk embeddings 생성
chunk_embeddings = sentence_model.encode(chunk_list)
chunk_embeddings.shape

(350, 768)

### 7.2. SentenceTransformers 검색 함수
- SentenceTransformers를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        result.append((i, doc_scores[i]))
    return result

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_sentence_transformer(sentence_model, chunk_embeddings, query)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학
---- score: 141.6308135986328 ----
수학
오늘날 수학은 자연과학, 공학뿐만 아니라, 경제학 등의 사회과학에서도 중요한 도구로 사용된다. 예를 들어, 정도의 차이는 있으나, 미적분학과 선형대수학은 자연과학과 공학, 경제학을 하는데에 필수적 과목으로 여겨지며, 확률론은 계량경제학에 응용된다. 통계학은 사회과학 이론에 근거를 마련하는데 필수적이다. 16세기에 갈릴레오 갈릴레이가 "자연이라는 책은 수학이라는 언어로 기록되어 있다."는 주장과 함께 물리학에 수학적 방법을 도입하였고, 17세기에 아이작 뉴턴이 고전 역학의 기본 물리학 법칙들을 수학적으로 기술하고 정립하여 물리학 이론에서 수학적 모델링은 필수적 요소가 되었다. 또한 이 시기는 과학적 방법이 정립되는 시기이기도 한데, 많은 과학적 현상들이 수학적 관계가 있음이 드러나면서 과학적 방법에도 수학은 중요한 역할을 하고 있다. 노벨 물리학상 수상자 유진 위그너는 그의 에세이 "The unreasonable effectiveness of mathematics in natural sciences"에서 인간 세상과 동떨어져있고 현실과 아무 관련이 없다고 여겨지던 수학 중 극히 일부는 뜻밖에도 자연과학과 연관성이 드러나고 과학이론에 효과적인 토대를 마련해 주는데에 대한 놀라움을 표현하였다. 예를 들어, 비유클리드 기하학과 3차원 이상의 임의의 차원에서 기하학을 탐구했던 미분 기하학은 당시에는 현실과 연관성을 가지지 않았으나 먼 훗날 일반상대성이론이 4차원 기하학을 필요로 함에 따라, 물리적 세상과 연관이 있음이 밝혀졌다. 또한 게이지이론, 양자장론 등에도 미분 기하학은 필수적이다.
또한 수학은 음악이나 미술 등 예술과도 관련이 있다. 피타고라스는 두 정수의 비율이 듣기 좋은 소리가 난다는 점을 가지고 피타고라스 음계를 만들었다. 중세시대에도 음악과 수학을 밀접하게 연관시켰으며 성 빅토르의 후고는 “음악은 조화다”라고 했고, 성 트론드의 루돌프는 “음악은 조화의 토대(ratio)다”

### 7.3. SLLM RAG with SentenceTransformers를 구현
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문에 대해서 관련된 문서를 SentenceTransformers를를 이용해서 5개 구합니다.
  - 질문과 함께 문서 5개를 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
sllm = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=sllm,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
# 프롬프트 생성 함수
def gen_prompt(pipe, chunk_list, query):
    messages = [
        {
            "role": "user",
            "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {chunk_list[4]}

문서4: {chunk_list[3]}

문서3: {chunk_list[2]}

문서2: {chunk_list[1]}

문서1: {chunk_list[0]}

질문: {query}"""
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_sentence_transformer(sentence_model, chunk_embeddings, query, top_n=5)
    chunk_list_result = [chunk_list[i] for i, score in result]
    result = gen_response(pipe, chunk_list_result, query)
    print(f'답변 > {result}\n\n')

질문 > 수학
답변 > **문서 1: 수학**

- 수학은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다.
- 현대 수학은 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 다른 학문들과 구별된다.
- 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서 핵심적인 역할을 하며 다양한 방식으로 응용된다.


**문서 2: 수학**

- 수학은 자연과학, 공학뿐만 아니라 경제학 등의 사회과학에서도 중요한 도구로 사용된다.
- 정도의 차이는 있으나, 미적분학과 선형대수학은 자연과학과 공학, 경제학을 하는데 필수적 과목으로 여겨지며, 확률론은 계량경제학에 응용된다.


**문서 3: 수학**

- 수학은 오늘날 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서 핵심적인 역할을 하며 다양한 방식으로 응용된다.
- 수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 고대 그리스어에서 유래되었다.


질문 > 


## 8. RAG 성능 확장 실습 (재시작 필요)
- CHUNK_FN의 전체 문서를 원하는 데이터로 구성해 보세요.
  - 예) 다른 위키 문서들
  - 예) 뉴스
  - 예) 책 또는 메뉴얼
- 프롬프트를 개선해 보세요.
- 'google/gemma-1.1-7b-it' 까지는 T4에서 겨우 돌아갑니다. 큰 LLM을 적용해 보세요.
- 다양한 방법으로 자신만의 RAG QA봇을 만들어 보세요.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.


### 나무위키 데이터 추가

In [ ]:
from datasets import load_dataset

dataset = load_dataset("heegyu/namuwiki")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
print(dataset["train"])

Dataset({
    features: ['title', 'text', 'contributors', 'namespace'],
    num_rows: 867024
})


In [ ]:
text_list = [example['text'] for example in dataset["train"]]

print(text_list[0])

#redirect 느낌표



In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [ ]:
# 빈 리스트 생성
data_list = []

# JSON 파일을 줄 단위로 읽어오기
with open(CHUNK_FN, 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 JSON 객체로 변환하여 리스트에 추가
        data = json.loads(line)
        data_list.append(data)

# 데이터 프레임으로 변환
df = pd.DataFrame(data_list)

chunk_list = df['chunk'].to_list()

# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in chunk_list]

java.lang.NullPointerException: java.lang.NullPointerException

나무위키 데이터의 양이 많아 추가 못함

### BM25

In [ ]:
bm25 = BM25Okapi(tokenized_chunks)

In [ ]:
def query_bm25(bm25, query, tokenizer, top_n=10):
    tokenized_query = tokenizer(query)
    # score 계산
    doc_scores = bm25.get_scores(tokenized_query)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        if doc_scores[i] > 0:
            result.append((i, doc_scores[i]))
    return result

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID2,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID2,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
def gen_prompt(pipe, chunk_list, query):
    # 문서 정보 문자열 생성
    doc_info = "\n\n".join([f"문서{i + 1}: {doc}" for i, doc in enumerate(chunk_list)])

    # 질문 문자열 생성
    question_info = f"질문: {query}"

    # 사용자 메시지 생성
    user_message = {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 문서들을 참고해서 질문에 대해 답변해 주세요:\n\n{doc_info}\n\n{question_info}"""

    }

    # 프롬프트 생성
    prompt = pipe.tokenizer.apply_chat_template([user_message],
                                                tokenize=False,
                                                add_generation_prompt=True)

    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_bm25(bm25, query, tokenizer, top_n=15)
    chunk_list_result = [chunk_list[i] for i, score in result]
    result = gen_response(pipe, chunk_list_result, query)
    print(f'답변 > {result}\n\n')

검색 > 수학과 과학의 차이가 뭐야?
답변 > **수학과 과학의 차이**

**1. 목표**

* **수학:** 수리적 모델과 공식을 개발하여 수학적 문제를 해결하고 예측을 내립니다.
* **과학:** 현상을 이해하고 이를 설명하고 예측하기 위해 실험과 관찰을 수행합니다.


**2. 접근 방식**

* **수학:** 추상적이고 일반화하여 새로운 문제에 적용할 수 있습니다.
* **과학:** 현실 세계에서 관찰된 데이터를 기반으로 이루어지며, 실험 결과를 통해 새로운 이론과 가설을 개발합니다.


**3. 결과**

* **수학:** 수리적 모델, 공식, 이론
* **과학:** 새로운 발견, 이론, 기술


**4. 적용**

* **수학:** 엔지니어링, 금융, 통계, 연구
* **과학:** 생명과학, 물리학, 화학, 생태학


**5. 가설 검증**

* **수학:** 가설을 수리적으로 검증하고 증명합니다.
* **과학:** 실험을 통해 가설을 검증하고 새로운 발견을 찾습니다.


**6. 일반화**

* **수학:** 일반화하여 새로운 상황에 적용할 수 있습니다.
* **과학:** 특정 실험 결과를 일반화하여 모든 상황에 적용하기 어렵습니다.


**7. 예측**

* **수학:** 데이터 분석을 통해 미래를 예측합니다.
* **과학:** 새로운 발견과 이론을 통해 미래를 예측합니다.


검색 > 수학과 과학의 차이점을 한문장으로 요약해줘
답변 > 수학은 추상적 수리 과정을 통해 수학적 개념과 정리법을 개발하고, 과학은 실제 현상을 이해하고 설명하기 위해 물리 법칙과 이론을 연구합니다.


검색 > KT에 대해서 설명해줘
답변 > **KT는 한국의 대표적인 통신기업으로, 한국전화기업(Korea Telecom)의 약자로, 한국에서 가장 큰 통신 기업입니다.** KT는 고정선, 이동통신, 인터넷, 데이터 등 다양한 통신 서비스를 제공하고 있으며, 국내외적으로 많은 고객과 사업자들과 협력관계를 유지하고 있습니다.

**KT의 주요 특징:**

* **국내 시장

### SentenceTransformers

In [ ]:
# SentenceTransformer 모델 생성
sentence_model = SentenceTransformer(SEARCH_MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# chunk embeddings 생성
chunk_embeddings = sentence_model.encode(chunk_list)
chunk_embeddings.shape

(350, 768)

In [ ]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        result.append((i, doc_scores[i]))
    return result

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
sllm = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID2,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID2,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=sllm,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
def gen_prompt(pipe, chunk_list, query):
    # 문서 정보 문자열 생성
    doc_info = "\n\n".join([f"문서{i + 1}: {doc}" for i, doc in enumerate(chunk_list)])

    # 질문 문자열 생성
    question_info = f"질문: {query}"

    # 사용자 메시지 생성
    user_message = {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 문서들을 참고해서 질문에 대해 답변해 주세요:\n\n{doc_info}\n\n{question_info}"""

    }

    # 프롬프트 생성
    prompt = pipe.tokenizer.apply_chat_template([user_message],
                                                tokenize=False,
                                                add_generation_prompt=True)

    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, chunk_list, query):
    prompt = gen_prompt(pipe, chunk_list, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_sentence_transformer(sentence_model, chunk_embeddings, query, top_n=5)
    chunk_list_result = [chunk_list[i] for i, score in result]
    result = gen_response(pipe, chunk_list_result, query)
    print(f'답변 > {result}\n\n')

질문 > 수학과 과학의 차이가 뭐야?
답변 > 문서에 제공된 정보에 따르면, 수학과 과학의 차이는 다음과 같습니다.

* **수학**은 수, 양, 구조, 공간, 변화 등의 개념을 다루고 추상적 대상을 탐구하며 엄밀한 논리에 근거하여 문제를 해결합니다.
* **과학**은 현실 세계를 이해하고 예측하기 위해 실험과 관찰을 수행하고 이론을 개발합니다.


질문 > 


## 9. 향후 성능 향상을 위한 방법에 대한 제안
- 문서 분할을 기계적 분할이 아닌 GPT-4를 이용해서 의미 기반으로 분할하는 방법도 있습니다.
- 검색 성능 향상을 위해서는 DPR 계열의 기술을 사용하길 추천합니다.
- 검색 성능 향상을 위해서 BM25와 DPR을 적절한 비율로 반영하면 더 좋은 성능을 얻을 수 있다는 연구 결과가 있습니다.
- 많은 문서를 사용하기 위해서 벡터DB를 사용하는 것을 추천드립니다.
- RAG 성능을 높이는 방법으로는 추가적인 fine-tuining을 해 보는 것을 추천드립니다.
- RAG를 학습을 위한 데이터는 예상되는 질문 및 관련 문서 쌍을 1,000개 이상 만들고 GPT-4에게 질문을 해보고 그 답변을 정답으로 사용해서, SLLM을 fine-tuining 해 보는 것도 좋은 방법입니다.

>대부분의 DPR 모델들이 한국어 지원을 하지 않아서 추후에 따로 한국어를 학습시켜서 수행해야함

### BM25 + DPR

In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [ ]:
# 빈 리스트 생성
data_list = []

# JSON 파일을 줄 단위로 읽어오기
with open(CHUNK_FN, 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 JSON 객체로 변환하여 리스트에 추가
        data = json.loads(line)
        data_list.append(data)

# 데이터 프레임으로 변환
df = pd.DataFrame(data_list)

chunk_list = df['chunk'].to_list()

# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in chunk_list]
print(tokenized_chunks[0])

['지미 카터', '제임스', '얼', '“', '지미', '”', '카터', '주니어', '(', ',', '1924', '년', '10월 1일', '~', ')', '민주당', '출신', '미국', '제', '39', '대', '대통령', '(', '1977', '-', '81', ')', '이', '.', '생애', '.', '어리', '시절', '.', '지미 카터', '조지', '아주', '섬터', '카운티', '플레인스', '마을', '태어나', '.', '조지아 공과대학교', '졸업', '하', '.', '그 후', '해군', '들어가', '전함', '·', '원자력', '·', '잠수함', '승무원', '일', '하', '.', '1953', '년', '미국 해군', '대위', '예편', '하', '이후', '땅콩', '·', '면', '화', '등', '가꾸', '많', '돈', '벌', '.', '그', '별명', '"', '땅콩', '농부', '"', '(', 'Peanut', 'Farmer', ')', '알리', '지', '.', '정계', '입문', '.', '1962', '년', '조지', '아주', '상원', '의원', '선거', '낙선', '하', ',', '그', '선거', '부정', '선거', '이', '입증', '하', '되', '당선', '되', ',', '1966', '년', '조지아 주', '지사', '선거', '낙선', '하', ',', '1970', '년', '조지아 주', '지사', '선거', '당선', '되', '.', '대통령', '되', '전', '조지', '아주', '상원', '의원', '두', '번', '연임', '하', ',', '1971', '년', '1975', '년', '조지아', '지사', '근무', '하', '.', '조지아 주', '지사', '지내', ',', '미국', '살', '흑인', '등용', '법', '내세우', '.', '대통령', '재임', '.', '1976', '년', '미', 

In [ ]:
bm25 = BM25Okapi(tokenized_chunks)

In [ ]:
from transformers import DPRContextEncoder, DPRQuestionEncoder, DPRReader

def initialize_dpr():
    # DPR 초기화
    context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
    reader = DPRReader.from_pretrained("facebook/dpr-reader-single-nq-base")
    return context_encoder, question_encoder, reader

In [ ]:
context_encoder, question_encoder, reader = initialize_dpr()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-reader-single-nq-base were not used when initializing DPRReader: ['span_predictor.encoder.bert_model.pooler.dense.bias', 'span_predictor.encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRReader from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRReader from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID2,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID2,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
def calculate_similarity_bm25(bm25, query, tokenizer):
    # BM25를 사용하여 문서별 유사도 계산
    tokenized_query = tokenizer(query)
    # score 계산
    doc_scores = bm25.get_scores(tokenized_query)

    return doc_scores

def calculate_similarity_dpr(query, chunk_list, context_encoder, question_encoder, reader):
    # DPR을 사용하여 문서별 유사도 계산
    with torch.no_grad():
        # Query를 인코딩
        encoded_question = question_encoder(query, return_tensors="pt")

        # 문서들을 인코딩
        encoded_contexts = context_encoder(chunk_list, return_tensors="pt", padding=True, truncation=True)

        # Query와 문서들 사이의 유사도 계산
        embeddings_question = encoded_question["hidden_states"][-1][:, 0, :]
        embeddings_contexts = encoded_contexts["hidden_states"][-1][:, 0, :]
        similarity_scores = reader(
            input_ids=encoded_question["input_ids"],
            context_input_ids=encoded_contexts["input_ids"],
            input_embeddings=embeddings_question,
            context_embeddings=embeddings_contexts,
        ).logits.squeeze().tolist()

    return similarity_scores